In [93]:
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score
import sklearn.metrics as metrics
from xgboost import XGBClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.preprocessing import OneHotEncoder

from  keras.wrappers.scikit_learn  import  KerasClassifier
from  sklearn.model_selection  import  GridSearchCV

import numpy as np
import pandas as pd

In [96]:
#part A)
df = pd.read_csv("https://raw.githubusercontent.com/FinalformSora/StockML/main/stocks%20v1%20-%20master%20sheet.csv")
df[0::24]
df.size

53520

In [97]:
#Data that  has been parsed and cleaned up
#Had to create a moving average for 50 day and 100 day. It requres calculation of previous days and finding the average
#increase is based on CLose-Open= if it equals positive its a true on increase
#AMD news is based on press release released by amd on their site. I had to manual add this in cause no site has this information parsed
#quarterly earning is also gather by searching the web for this release date
df

,Date,Stock name,Open,High,Low,Close*,Adj Close**,Volume,average volume per stock,Higher or lower then average volume,...,Volitity (1 or 0),mv50,Higher MV50,Higher than MVA50,MV100,Higher MVA100,Higher than MVA100,EMA 50,Higher then EMVA,Higher than EMVA 1
0,44294.0,S&P500,4089.95,4098.19,4082.54,4097.17,4097.17,3.901910e+09,4.358682e+09,No,...,0,3906.56,Yes,1,3893.72,Yes,1,3909.06,No,0
1,44293.0,S&P500,4074.29,4083.13,4068.31,4079.95,4079.95,4.112640e+09,4.358682e+09,No,...,0,3901.61,Yes,1,3888.90,Yes,1,3901.39,No,0
2,44292.0,S&P500,4075.57,4086.23,4068.14,4073.94,4073.94,4.027880e+09,4.358682e+09,No,...,0,3897.12,Yes,1,3883.46,Yes,1,3894.10,No,0
3,44291.0,S&P500,4034.44,4083.42,4034.44,4077.91,4077.91,3.999760e+09,4.358682e+09,No,...,0,3892.47,Yes,1,3878.31,Yes,1,3886.76,No,0
4,44287.0,S&P500,3992.78,4020.63,3992.78,4019.87,4019.87,4.151240e+09,4.358682e+09,No,...,0,3887.97,Yes,1,3873.07,Yes,1,3878.96,No,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2225,44286.0,CokeCola,294.35,297.74,288.59,288.78,288.78,3.410000e+04,4.625314e+04,No,...,0,276.98,Yes,1,272.16,Yes,1,280.08,No,0
2226,44287.0,CokeCola,287.78,289.94,286.95,289.17,289.17,2.280000e+04,4.625314e+04,No,...,0,277.64,Yes,1,272.73,Yes,1,280.43,No,0
2227,44291.0,CokeCola,288.15,298.87,288.00,295.19,295.19,3.000000e+04,4.625314e+04,No,...,0,278.39,Yes,1,273.27,Yes,1,281.01,No,0
2228,44292.0,CokeCola,297.62,301.77,296.14,296.36,296.36,2.140000e+04,4.625314e+04,No,...,0,279.07,Yes,1,273.86,Yes,1,281.61,No,0


In [92]:
df.value_counts()
##cleanup_StockName = {"Stock name": {"CokeCola": 1, "S&P500": 2,"Apple": 3,"Gamestop":4}}
##df = df.replace(cleanup_StockName)
##df.head()

AttributeError: 'DataFrame' object has no attribute 'value_counts'

In [ ]:
df

In [84]:
#feature_cols = ['Volume','50 moving day average (1 or 0)','100 day moving average (1 or 0)','AMD News (1 or 0)','Quarterly earnings (1 or 0)']


feature_cols = ['Stock name','Volume avg','Volitity (1 or 0)']
X = df[feature_cols]
X.head()



,Stock name,Volume avg,Volitity (1 or 0)
0,2,0,0
1,2,0,0
2,2,0,0
3,2,0,0
4,2,0,0


In [85]:
#created the label based on if the stock increased or did not increase

In [98]:
y = df['higher closing 0 or 1']
y[::25]

0       1
25      0
50      0
75      1
100     1
       ..
2125    0
2150    1
2175    0
2200    1
2225    0
Name: higher closing 0 or 1, Length: 90, dtype: int64

In [99]:
#calling all the classifers
my_logreg = LogisticRegression()
my_decisiontree = DecisionTreeClassifier()
knn = KNeighborsClassifier(n_neighbors=4)
my_RandomForest = RandomForestClassifier()
my_linreg = LinearRegression()
my_AdaBoost = AdaBoostClassifier(n_estimators = 29,random_state=2)
my_XGBoost = XGBClassifier(n_estimators = 29,random_state=2)


In [100]:
#seting up train set
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=5)

In [102]:
# training sets

knn.fit(X_train, y_train)
my_logreg.fit(X_train, y_train)
my_RandomForest.fit(X_train, y_train)
my_decisiontree.fit(X_train, y_train)
my_linreg.fit(X_train, y_train)
my_AdaBoost.fit(X_train,y_train)
##my_XGBoost.fit(X_train,y_train)


AdaBoostClassifier(n_estimators=29, random_state=2)

In [103]:
#Training
y_predict_lor = my_logreg.predict(X_test)
y_predict = knn.predict(X_test)
y_predict_dt = my_decisiontree.predict(X_test)
y_predict_rt = my_RandomForest.predict(X_test)
y_predict_LR = my_linreg.predict(X_test)
y_predict_AB = my_AdaBoost.predict(X_test)
y_predict_XB = my_XGBoost.predict(X_test)


In [104]:
#accuracy of the perdiction
from sklearn.metrics import accuracy_score

score_lor = accuracy_score(y_test, y_predict_lor)
score_dt = accuracy_score(y_test, y_predict_dt)
accuracy = accuracy_score(y_test, y_predict)
score_rt = accuracy_score(y_test,y_predict_rt)
score_AB = accuracy_score(y_test, y_predict_AB)
score_XB = accuracy_score(y_test, y_predict_XB)

y_predict_LR = my_linreg.predict(X_test)
mse = metrics.mean_squared_error(y_test, y_predict_LR)
rmse = np.sqrt(mse)
print("Linear Regression",rmse)
print('LogisticRegression: ',score_lor)
print("Decision Tree: ",score_dt)
print("Random Forest: ",score_rt)
print("knn: ",accuracy)
print("AdaBoost: ",score_AB)
print("XGBoost: ",score_XB)


Linear Regression 0.5003350862794591
LogisticRegression:  0.5017921146953405
Decision Tree:  0.5268817204301075
Random Forest:  0.5268817204301075
knn:  0.45340501792114696
AdaBoost:  0.517921146953405
XGBoost:  0.5268817204301075


In [105]:
#Cross validation of all Classifers

In [106]:
accuracy_list = cross_val_score(my_logreg, X, y, cv=10, scoring='accuracy')
accuracy_cv = accuracy_list.mean()
print("Logistic Regression: ",accuracy_cv)

Logistic Regression:  0.3834080717488789


In [107]:
accuracy_list = cross_val_score(my_RandomForest, X, y, cv=10, scoring='accuracy')
accuracy_cv = accuracy_list.mean()

print("Random Forest: ",accuracy_cv)

Random Forest:  0.4475336322869955


In [108]:
accuracy_list = cross_val_score(knn, X, y, cv=10, scoring='accuracy')
accuracy_cv = accuracy_list.mean()

print("KNN: ",accuracy_cv)


KNN:  0.4502242152466368


In [109]:
accuracy_list = cross_val_score(my_decisiontree, X, y, cv=10, scoring='accuracy')
accuracy_cv = accuracy_list.mean()

print("decisiontree: ",accuracy_cv)

decisiontree:  0.42735426008968613


In [110]:
mse_list = cross_val_score(my_linreg, X, y, cv=10, scoring='neg_mean_squared_error')
mse_list_positive = -mse_list
rmse_list = np.sqrt(mse_list_positive)
print("Linear Regression: ",rmse_list.mean())


Linear Regression:  0.5029123528781592


In [111]:
accuracy_list = cross_val_score(my_AdaBoost, X, y, cv=10, scoring='accuracy')
accuracy_ab = accuracy_list.mean()

print("Adaboost : ",accuracy_ab)

Adaboost :  0.45739910313901344


In [112]:
accuracy_list = cross_val_score(my_XGBoost, X, y, cv=10, scoring='accuracy')
accuracy_XG = accuracy_list.mean()

print("XGBoost: ",accuracy_XG)

[19:52:34] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[19:52:34] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[19:52:34] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[19:52:34] WARNING: C:/Users/Administrator/workspace/xgboost-win64_rel

C:\Users\Justin Nim\anaconda3\lib\site-packages\xgboost\sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


In [113]:
from keras.utils import np_utils
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from sklearn.neural_network import MLPClassifier

from sklearn import preprocessing

my_ANN = MLPClassifier(hidden_layer_sizes=(30,), activation= 'logistic', 
                       solver='adam', alpha=1, random_state=1, 
                       learning_rate_init = 0.02, verbose=True)

from sklearn.model_selection import cross_val_score
accuracy_list = cross_val_score(my_ANN, preprocessing_features, y, cv=10, scoring='accuracy')

NameError: name 'preprocessing_features' is not defined

In [ ]:
from sklearn.model_selection import GridSearchCV
import numpy as np

seed = 2
np.random.seed(seed)

model = KerasClassifier(build_fn = model_creator, verbose=2)


batch_size = [30 , 50 , 100 ]
epochs = [10 , 15 , 20]

param_grid = dict(batch_size=batch_size, epochs=epochs)
grid = GridSearchCV(model, param_grid, cv=10, scoring='neg_log_loss')


In [ ]:
my_grid = grid.fit(X_train, y_train)

print("best: %f with %s" % (my_grid.best_score_, my_grid.best_params_))